<a href="https://colab.research.google.com/github/Olhaau/fl-official-statistics-addon/blob/main/03_JW_tuned_insurance_federated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Insurance - a Federated Learning Use Case.

This notebook contains Federated Learning.

-- **more tba** --

## Setup
---

In [ ]:
# Is a repo-clone and installs needed (e.g. in colabs)? 
need_clone_install = True

### Pull Repo

In [ ]:
if need_clone_install:
    import os
    
    # rm repo from gdrive
    if os.path.exists("fl-official-statistics-addon"):
      %rm -r fl-official-statistics-addon

    # clone
    !git clone https://github.com/Olhaau/fl-official-statistics-addon
    %cd fl-official-statistics-addon

    # pull (the currenct version of the repo)
    !git pull

Cloning into 'fl-official-statistics-addon'...
remote: Enumerating objects: 835, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 835 (delta 11), reused 8 (delta 8), pack-reused 818
Receiving objects: 100% (835/835), 33.09 MiB | 16.23 MiB/s, done.
Resolving deltas: 100% (371/371), done.
/content/fl-official-statistics-addon
Already up to date.


### Installs

In [ ]:
if need_clone_install: 
  !pip install --quiet nest_asyncio
  !pip install --quiet tensorflow_federated
  !pip install --quiet tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 KB 28.0 MB/s eta 0:0

### Imports

In [ ]:
import collections
import numpy as np
import pandas as pd
import tqdm
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow_addons.metrics import RSquare
import tensorflow_addons as tfa
import nest_asyncio
from sklearn.metrics import r2_score

np.random.seed(0)

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


## Ingest and Split the Data

In [ ]:
df = pd.read_csv("output/data/insurance-clean.csv", index_col = 0)
df.head()

,age,sex,bmi,children,smoker,region,charges,region0,region1,region2,region3
0,0.021739,0.0,0.321227,0.0,1.0,southwest,16884.92400,0.0,0.0,0.0,1.0
1,0.000000,1.0,0.479150,0.2,0.0,southeast,1725.55230,0.0,0.0,1.0,0.0
2,0.217391,1.0,0.458434,0.6,0.0,southeast,4449.46200,0.0,0.0,1.0,0.0
3,0.326087,1.0,0.181464,0.0,0.0,northwest,21984.47061,0.0,1.0,0.0,0.0
4,0.304348,1.0,0.347592,0.0,0.0,northwest,3866.85520,0.0,1.0,0.0,0.0


In [ ]:
NUM_CLIENTS = 4
NUM_EPOCHS = 50
BATCH_SIZE = 128
SHUFFLE_BUFFER = 20
PREFETCH_BUFFER = 5
NUM_ROUNDS = 50
RUN_NAME = f'0,8-3({NUM_ROUNDS})-{NUM_EPOCHS}-epochs-{BATCH_SIZE}-batch-WithRegion/'

In [ ]:
syn_samples_per_region = 1000

def get_dataset_for_region(dataset, region_index, test_size_per_region=20):
    """Min-max scale and return data for a single, given region. The scaler must be fitted before.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param region_index: The index number of the region to return
    :type region_index: int
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: The dataset specific for the defined region, the test values, the test labels
    :rtype: tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series
    """
    region_ds = dataset[dataset['region'] == region_index]
    region_ds = region_ds.drop(columns=['region'])
    len = region_ds.shape[0]

    # The scaling into [0, 1] is not necessary anymore, it happens when the data loads already
    # region_ds[['age', 'bmi', 'children']] = scaler.transform(region_ds[['age', 'bmi', 'children']])

    X_test = region_ds.head(test_size_per_region)
    y_test = X_test.pop('charges')

    X_train = region_ds.tail(len - test_size_per_region)
    y_train = X_train.pop('charges')

    fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

    return (
        fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER),
        (X_test, y_test)
    )

In [ ]:
# Create test and train sets and put them into random_client_ds, use four clients which are independent of the region
def get_dataset_random_region(dataset, num_clients=4, test_size_per_region=20):
    """Creates a list with client datasets independent of the region.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param num_clients: the number of clients create (equal big datasets per client), default value is 4 clients
    :type num_clients: int, optional
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: List of the prepared dataset with one entry per region, the test values and labels for each region
    :rtype: List of (tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series)"""
    size_of_client_ds = int(dataset.shape[0] / num_clients)

    dataset_to_split = dataset.copy()
    dataset_to_split.pop("region")
    random_client_ds = []
    for i in range(num_clients):
        sampled = dataset_to_split.sample(n=size_of_client_ds)
        dataset_to_split.drop(sampled.index)

        X_test = sampled.head(test_size_per_region)
        y_test = X_test.pop('charges')

        X_train = sampled.tail(size_of_client_ds - test_size_per_region)
        y_train = X_train.pop('charges')

        fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

        train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)
        test_set = (X_test, y_test)

        random_client_ds.append((train_set, test_set))

    return random_client_ds

In [ ]:
# Training data for clients with regional data (each client one region)
test_size_per_region = 20
regions = ['region0', 'region1', 'region2', 'region3']

federated_insurance_data = [
    get_dataset_for_region(df.drop(regions, axis=1), i, test_size_per_region=test_size_per_region)
    for i in range(NUM_CLIENTS-1)]
federated_insurance_data

[(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 5), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (Empty DataFrame
   Columns: [age, sex, bmi, children, smoker]
   Index: [],
   Series([], Name: charges, dtype: float64))),
 (<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 5), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (Empty DataFrame
   Columns: [age, sex, bmi, children, smoker]
   Index: [],
   Series([], Name: charges, dtype: float64))),
 (<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 5), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (Empty DataFrame
   Columns: [age, sex, bmi, children, smoker]
   Index: [],
   Series([], Name: charges, dtype: float64)))]

In [ ]:
# Training data for clients with regional independent data
random_client_ds = get_dataset_random_region(df, num_clients=NUM_CLIENTS, test_size_per_region=test_size_per_region)

### Code Tests

In [ ]:
print("splitted datasets: {}".format(len(federated_insurance_data)))
print("------------------------------")
# Q: why range(NUM_CLIENTS-1) and, thus, only 3 clients?

# Q: how to output? -> does not work or nothing in it?
# -> i think its empty
list(federated_insurance_data[0][0].as_numpy_iterator())[:10]

splitted datasets: 3
------------------------------


[]

#### Show Tensor

In [ ]:
# Create a Tensor Object
X_train = df.iloc[:,0:4]
y_train = df['charges']
df2 = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))
df2

<_TensorSliceDataset element_spec=(TensorSpec(shape=(4,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>

In [ ]:
# Show Tensor Object
# S. https://stackoverflow.com/questions/62436302/extract-target-from-tensorflow-prefetchdataset
list(df2.as_numpy_iterator())[:10]

[(array([0.02173913, 0.        , 0.3212268 , 0.        ]), 16884.924),
 (array([0.        , 1.        , 0.47914985, 0.2       ]), 1725.5523),
 (array([0.2173913 , 1.        , 0.45843422, 0.6       ]), 4449.462),
 (array([0.32608696, 1.        , 0.18146355, 0.        ]), 21984.47061),
 (array([0.30434783, 1.        , 0.34759214, 0.        ]), 3866.8552),
 (array([0.2826087 , 0.        , 0.26311542, 0.        ]), 3756.6216),
 (array([0.60869565, 0.        , 0.47027172, 0.2       ]), 8240.5896),
 (array([0.41304348, 0.        , 0.31692225, 0.6       ]), 7281.5056),
 (array([0.41304348, 1.        , 0.37315039, 0.4       ]), 6406.4107),
 (array([0.91304348, 0.        , 0.26580576, 0.        ]), 28923.13692)]

## Model Builder

In [ ]:
from keras.layers import ReLU
from keras.models import Sequential
from keras.layers import Dense

def create_keras_model(
  input_features = 9
  ):
  model = Sequential()
  model.add(Dense(40, input_dim=9))
  model.add(ReLU())
  model.add(Dense(40))
  model.add(ReLU())
  model.add(Dense(20))
  model.add(ReLU())
  model.add(Dense(1))
  return model


# A helper function for federated learning
def model_fn(    
  input_features = 9):

  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model(
          input_features = input_features
  )
  return tff.learning.models.from_keras_model(
      keras_model,
      # without region: 
      #input_spec = federated_insurance_data[0][0].element_spec,
      input_spec = random_client_ds[0][0].element_spec,
      loss = tf.keras.losses.MeanAbsoluteError(),
      #loss = tf.keras.losses.MeanSquaredError(),
      metrics = [tf.keras.metrics.MeanAbsoluteError()
      #metrics = [tf.keras.losses.MeanSquaredError()
      #,tfa.metrics.RSquare()
      ]
  )

# Helper functions for different features as input
def model_fn_5():
    return model_fn(5)

def model_fn_5_mod():
  return model_fn(5, initializer = 'glorot_uniform', activation = 'relu')

def model_fn_9():
  return model_fn(9)

def model_fn_9_mod():
  return model_fn(9, initializer = 'glorot_uniform', activation = 'relu')

## Federated Learning

**bold text**### Tuned Model

#### Create Learning Process

In [ ]:
# Create iterative learning process which will perform the federated learning
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn_9,
    client_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),
    server_optimizer_fn=lambda: tf.optimizers.Adam(learning_rate = .05),)

In [ ]:
# The initial setup of the learning process
print(iterative_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,40],
      float32[40],
      float32[40,40],
      float32[40],
      float32[40,20],
      float32[20],
      float32[20,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[9,40],
    float32[9,40],
    float32[40],
    float32[40],
    float32[40,40],
    float32[40,40],
    float32[40],
    float32[40],
    float32[40,20],
    float32[40,20],
    float32[20],
    float32[20],
    float32[20,1],
    float32[20,1],
    float32[1],
    float32[1]
  >
>@SERVER)


#### Train

In [ ]:
state = iterative_process.initialize()

In [ ]:
# Train the federated model with random clients
for round_num in tqdm.tqdm(range(1, NUM_ROUNDS)):
    result = iterative_process.next(state, [f[0] for f in random_client_ds])
    state = result.state
    metrics = result.metrics
    for name, value in metrics['client_work']['train'].items():
        tf.summary.scalar(name, value, step=round_num)

100%|██████████| 149/149 [02:53<00:00,  1.17s/it]


In [ ]:
result.metrics

OrderedDict([('distributor', ()),
             ('client_work',
              OrderedDict([('train',
                            OrderedDict([('mean_absolute_error', 2456.5483),
                                         ('loss', 2456.5493),
                                         ('num_examples', 6280),
                                         ('num_batches', 628)]))])),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', OrderedDict([('update_non_finite', 0)]))])

In [ ]:
result.state
# all weights are zero
# -> nothing is learned 

LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[array([[-0.1741766 , -0.65945995,  0.24910524,  1.7127984 ,  1.0322018 ,
         3.685859  , -0.2952447 , -3.2420151 ,  0.34301487, -0.09678213,
         0.10899331,  1.8386388 ,  2.126536  ,  4.0187216 ,  0.09266797,
        -0.5785229 , -0.61664355,  0.06203857,  0.09016142, -0.06289557,
         3.6999717 ,  4.2809653 ,  0.8165719 ,  2.552904  ,  1.306948  ,
         2.1914408 , -0.04340182,  0.01833579, -0.21538445,  2.882945  ,
        -0.22525805,  0.9124447 ,  3.391181  ,  4.2296424 ,  1.5336322 ,
         0.29523712,  0.13256262,  1.0600549 , -0.23340385,  2.1941328 ],
       [-0.75236374,  0.21465611, -0.1894954 , -1.2954636 , -2.2063828 ,
         0.22172844,  0.11475755, -0.8576769 , -0.74287486, -0.3016841 ,
         0.09647838, -1.6223379 , -0.5769636 ,  0.24122943,  0.05421387,
        -0.22627658, -0.24306634,  0.0685216 ,  0.12015455,  0.10808164,
        -0.31444538, -0.21072882, -1.5016952 , -0.95643

#### Evaluate

In [ ]:
# Create the test data for model evaluation
X_test = pd.concat([f[1][0] for f in random_client_ds])
y_test = pd.concat([f[1][1] for f in random_client_ds])

test_sets = [tf.data.Dataset.from_tensor_slices(
    (tf.convert_to_tensor(np.expand_dims(el[1][0], axis=0)), 
    tf.convert_to_tensor(np.expand_dims(el[1][1], axis=0)))) 
    for el in random_client_ds]

In [ ]:
# Model evaluation
evaluation = tff.learning.build_federated_evaluation(model_fn_9)
# print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(state)
train_metrics = evaluation(model_weights, test_sets)
train_metrics

<ipython-input-48-552d4a23315f>:2: DeprecationWarning: `tff.learning.build_federated_evaluation` is deprecated, use `tff.learning.algorithms.build_fed_eval` instead.
  evaluation = tff.learning.build_federated_evaluation(model_fn_9)


OrderedDict([('eval',
              OrderedDict([('mean_absolute_error', 2280.2578),
                           ('loss', 2280.2578),
                           ('num_examples', 80),
                           ('num_batches', 4)]))])

In [ ]:
# Create model from training results and evaluate
model = create_keras_model(input_features = 9)
model_weights.assign_weights_to(model)
model.compile(
    loss=tf.losses.mae,
    # loss=tf.losses.mean_squared_error,
    optimizer=tf.optimizers.Adam(),
    metrics=["mae", 'mean_squared_error']
)
# The evaluation results, for technical reasons the metrics_names is called afterwards. However, its order fits to the results
print(model.evaluate(X_test, y_test))
print(model.metrics_names)

3/3 [==============================] - 0s 8ms/step - loss: 2280.2578 - mae: 2280.2578 - mean_squared_error: 27328934.0000
[2280.2578125, 2280.2578125, 27328934.0]
['loss', 'mae', 'mean_squared_error']


Results federated:

fl-loss (mae): 2280.2578 
🚀